In [14]:
import librosa
import glob
import pandas as pd
import numpy as np
import os
import tensorflow
import h5py
import pandas as pd

appended_data = []

for file in glob.glob('/Users/zhongxur/Desktop/EE599/AOL-user-ct-collection/data_full/*.txt'):
    Single_txt = pd.read_csv(file,delimiter="\t")
    appended_data.append(Single_txt)
    
data_full = pd.concat(appended_data)


data_full.to_csv('Full_data.csv')

v = data_full.AnonID.value_counts()
Data_100 = data_full[data_full.AnonID.isin(v.index[v.gt(100)])]


Data_100.to_csv('data_100Entry.csv')

# data_filtered['Query'] = str(data_filtered['Query'])

Cancer_key = pd.read_csv('/Users/zhongxur/Desktop/EE599/AOL-user-ct-collection/Cancer_Keywords.csv')

Alcohol_key =pd.read_csv('/Users/zhongxur/Desktop/EE599/AOL-user-ct-collection/Alcoholic_Keywords.csv')

Pregnancy_key = pd.read_csv('/Users/zhongxur/Desktop/EE599/AOL-user-ct-collection/Pregnant_Keywords.csv')

Keywords = pd.concat([Cancer_key,Alcohol_key])
Keywords = pd.concat([Keywords,Pregnancy_key])

In [12]:
import pandas as pd
import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from pprint import pprint
import numpy as np
import nltk
nltk.download('wordnet')


stemmer = SnowballStemmer('english')
 
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
 
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [26]:
documents = Cancer_key['Keyword']


processed_docs = documents.map(preprocess)
processed_docs[:10]

dictionary = gensim.corpora.Dictionary(processed_docs)


dictionary.filter_extremes(no_below=1, no_above=0.01, keep_n=1000)
len(dictionary)

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

bow_doc_0 = bow_corpus[0]
for i in range(len(bow_doc_0)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_0[i][0], 
                                               dictionary[bow_doc_0[i][0]], 
bow_doc_0[i][1]))

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]


lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=20, id2word=dictionary, passes=2, workers=2)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
    
    

Topic: 0 
Words: 0.144*"form" + 0.112*"patient" + 0.112*"introduct" + 0.081*"address" + 0.081*"fact" + 0.049*"sit" + 0.049*"leukemia" + 0.002*"site" + 0.002*"definit" + 0.002*"info"
Topic: 1 
Words: 0.181*"diagnosi" + 0.051*"articl" + 0.051*"process" + 0.051*"tropic" + 0.051*"paragraph" + 0.051*"colon" + 0.051*"search" + 0.051*"prevent" + 0.034*"statist" + 0.018*"care"
Topic: 2 
Words: 0.100*"stag" + 0.100*"donat" + 0.075*"name" + 0.038*"british" + 0.038*"magazin" + 0.038*"sick" + 0.038*"unit" + 0.038*"recurr" + 0.038*"pathophysiolog" + 0.038*"growth"
Topic: 3 
Words: 0.114*"oncolog" + 0.086*"site" + 0.072*"كونكر" + 0.044*"impact" + 0.044*"characterist" + 0.044*"factor" + 0.044*"offici" + 0.044*"ribbon" + 0.044*"pink" + 0.044*"bodi"
Topic: 4 
Words: 0.092*"prevent" + 0.092*"news" + 0.092*"skin" + 0.056*"basic" + 0.056*"jewelri" + 0.056*"gift" + 0.056*"tratamiento" + 0.056*"page" + 0.020*"walk" + 0.003*"stage"
Topic: 5 
Words: 0.091*"site" + 0.076*"rat" + 0.076*"articl" + 0.046*"discuss

In [62]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
 
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.212*"type" + 0.053*"differ" + 0.049*"symptom" + 0.045*"tumor" + 0.041*"malign" + 0.034*"breast" + 0.033*"lung" + 0.032*"site" + 0.026*"treatment" + 0.019*"altern"
Topic: 1 Word: 0.093*"organ" + 0.046*"human" + 0.039*"awar" + 0.037*"breast" + 0.037*"oncolog" + 0.031*"note" + 0.029*"research" + 0.028*"american" + 0.023*"chariti" + 0.022*"short"
Topic: 2 Word: 0.128*"research" + 0.087*"institut" + 0.060*"nation" + 0.047*"inform" + 0.040*"websit" + 0.039*"number" + 0.029*"general" + 0.025*"support" + 0.024*"breast" + 0.022*"lung"
Topic: 3 Word: 0.071*"societi" + 0.056*"foundat" + 0.040*"breast" + 0.038*"statist" + 0.038*"chariti" + 0.031*"ill" + 0.027*"sign" + 0.023*"symptom" + 0.020*"american" + 0.019*"inform"
Topic: 4 Word: 0.055*"stage" + 0.043*"breast" + 0.041*"cell" + 0.041*"termin" + 0.040*"kind" + 0.029*"stag" + 0.029*"name" + 0.029*"cancel" + 0.025*"month" + 0.025*"diseas"
Topic: 5 Word: 0.136*"diseas" + 0.044*"center" + 0.042*"wikipedia" + 0.035*"differ" + 0.032*"

In [25]:
documents = Pregnancy_key['Keyword']


processed_docs = documents.map(preprocess)
processed_docs[:10]

dictionary = gensim.corpora.Dictionary(processed_docs)


dictionary.filter_extremes(no_below=2, no_above=0.01, keep_n=1000)
len(dictionary)

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

bow_doc_0 = bow_corpus[0]
for i in range(len(bow_doc_0)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_0[i][0], 
                                               dictionary[bow_doc_0[i][0]], 
bow_doc_0[i][1]))

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]


lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.142*"healthi" + 0.142*"want" + 0.120*"morn" + 0.120*"sick" + 0.120*"relat" + 0.075*"help" + 0.029*"educ" + 0.007*"ladi" + 0.007*"second" + 0.007*"miscarriag"
Topic: 1 
Words: 0.146*"quick" + 0.146*"pictur" + 0.146*"period" + 0.123*"prenat" + 0.123*"second" + 0.030*"mother" + 0.030*"weight" + 0.007*"miscarriag" + 0.007*"ladi" + 0.007*"natur"
Topic: 2 
Words: 0.226*"fact" + 0.202*"plan" + 0.080*"miscarriag" + 0.080*"second" + 0.080*"help" + 0.031*"prenat" + 0.031*"diabet" + 0.007*"natur" + 0.007*"video" + 0.007*"cycl"
Topic: 3 
Words: 0.169*"mother" + 0.128*"labor" + 0.128*"fast" + 0.128*"detail" + 0.108*"deliveri" + 0.067*"sit" + 0.026*"chanc" + 0.026*"effect" + 0.006*"prenat" + 0.006*"natur"
Topic: 4 
Words: 0.250*"step" + 0.197*"natur" + 0.143*"diabet" + 0.089*"troubl" + 0.008*"miscarriag" + 0.008*"second" + 0.008*"cycl" + 0.008*"plan" + 0.008*"ladi" + 0.008*"stag"
Topic: 5 
Words: 0.139*"matern" + 0.118*"tri" + 0.118*"ladi" + 0.118*"risk" + 0.118*"educ" + 0.118*"ef

In [20]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
 
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.085*"common" + 0.084*"get" + 0.080*"video" + 0.064*"inform" + 0.058*"childbirth" + 0.051*"quick" + 0.048*"educ" + 0.048*"relat" + 0.044*"risk" + 0.042*"mom"
Topic: 1 Word: 0.119*"best" + 0.108*"fertil" + 0.104*"woman" + 0.068*"plan" + 0.061*"help" + 0.058*"possibl" + 0.053*"earliest" + 0.041*"tri" + 0.032*"cycl" + 0.029*"diet"
Topic: 2 Word: 0.119*"trimest" + 0.116*"month" + 0.102*"preg" + 0.095*"calendar" + 0.073*"miscarriag" + 0.072*"calcul" + 0.055*"earliest" + 0.052*"matern" + 0.040*"fertil" + 0.037*"stage"
Topic: 3 Word: 0.120*"women" + 0.107*"class" + 0.070*"childbirth" + 0.068*"ladi" + 0.066*"natur" + 0.061*"inform" + 0.051*"websit" + 0.050*"detail" + 0.049*"mother" + 0.030*"infertil"
Topic: 4 Word: 0.202*"birth" + 0.122*"stage" + 0.079*"weight" + 0.074*"develop" + 0.047*"gain" + 0.039*"chanc" + 0.039*"healthi" + 0.032*"plan" + 0.027*"risk" + 0.015*"exercis"
Topic: 5 Word: 0.156*"date" + 0.139*"get" + 0.083*"calcul" + 0.058*"fast" + 0.055*"babi" + 0.051*"have" +

In [28]:
documents = Alcohol_key['Keyword']


processed_docs = documents.map(preprocess)
processed_docs[:10]

dictionary = gensim.corpora.Dictionary(processed_docs)


dictionary.filter_extremes(no_below=2, no_above=0.01, keep_n=1000)
len(dictionary)

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

bow_doc_0 = bow_corpus[0]
for i in range(len(bow_doc_0)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_0[i][0], 
                                               dictionary[bow_doc_0[i][0]], 
bow_doc_0[i][1]))

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]


lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=20, id2word=dictionary, passes=2, workers=2)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.249*"quiz" + 0.249*"articl" + 0.140*"high" + 0.140*"statist" + 0.031*"program" + 0.004*"classifi" + 0.004*"harm" + 0.004*"drinker" + 0.004*"peopl" + 0.004*"constitut"
Topic: 1 
Words: 0.277*"test" + 0.238*"support" + 0.200*"teenag" + 0.006*"physic" + 0.006*"program" + 0.006*"clinic" + 0.006*"articl" + 0.006*"high" + 0.006*"peopl" + 0.006*"bodi"
Topic: 2 
Words: 0.173*"poison" + 0.173*"drug" + 0.173*"common" + 0.173*"treat" + 0.039*"etoh" + 0.039*"alchol" + 0.005*"high" + 0.005*"clinic" + 0.005*"peopl" + 0.005*"classifi"
Topic: 3 
Words: 0.309*"alchol" + 0.157*"think" + 0.096*"articl" + 0.096*"classifi" + 0.035*"rehabilit" + 0.035*"constitut" + 0.035*"everyday" + 0.035*"drug" + 0.005*"issu" + 0.005*"program"
Topic: 4 
Words: 0.346*"disord" + 0.065*"center" + 0.065*"constitut" + 0.065*"bodi" + 0.065*"substanc" + 0.008*"articl" + 0.008*"clinic" + 0.008*"peopl" + 0.008*"danger" + 0.008*"inform"
Topic: 5 
Words: 0.362*"sever" + 0.167*"determin" + 0.102*"stori" + 0.102*"di

In [67]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
 
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.224*"sign" + 0.099*"excess" + 0.098*"consumpt" + 0.061*"symptom" + 0.055*"chronic" + 0.050*"consid" + 0.042*"treat" + 0.035*"medic" + 0.034*"test" + 0.029*"awar"
Topic: 1 Word: 0.172*"definit" + 0.131*"make" + 0.098*"anonym" + 0.094*"quit" + 0.055*"clinic" + 0.050*"drink" + 0.046*"inform" + 0.035*"function" + 0.020*"depend" + 0.020*"treatment"
Topic: 2 Word: 0.168*"depend" + 0.122*"symptom" + 0.105*"help" + 0.104*"know" + 0.056*"program" + 0.042*"physic" + 0.041*"sever" + 0.037*"rehabilit" + 0.035*"center" + 0.031*"abus"
Topic: 3 Word: 0.137*"stop" + 0.104*"health" + 0.063*"danger" + 0.063*"abus" + 0.052*"substanc" + 0.049*"diseas" + 0.047*"drink" + 0.042*"what" + 0.038*"relat" + 0.037*"support"
Topic: 4 Word: 0.229*"abus" + 0.166*"consid" + 0.094*"defin" + 0.072*"treatment" + 0.046*"drug" + 0.044*"person" + 0.039*"consequ" + 0.036*"etoh" + 0.035*"caus" + 0.027*"stori"
Topic: 5 Word: 0.325*"drink" + 0.104*"abus" + 0.076*"tell" + 0.047*"make" + 0.044*"problem" + 0.042*"